In [24]:
import pandas as pd
import numpy as np
import boto3
s3 = boto3.client('s3')
bucket_name = 'sagemaker-studio-905418013525-nvxe84zgs6'
file_path = 'Labrado/alllabs1000adm/Labitemlist.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
Labitemlist = pd.read_csv(obj['Body'], header=None,names=['labitem', 'cnt'], dtype=int)

file_path = 'Labrado/alllabs1000adm/unique_adm_year_day.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
unique_adm_year_day = pd.read_csv(obj['Body'], header=None,names=['hadm_id', 'chartyear', 'day','rowinx'], dtype=int)

In [70]:
Labitemlist.shape

(375, 2)

In [27]:
unique_adm_year_day.head()

,hadm_id,chartyear,day,rowinx
0,20000019,2159,0,0
1,20000019,2159,1,1
2,20000019,2159,2,2
3,20000024,2151,0,3
4,20000034,2174,0,4


In [15]:
unique_hadm_by_year = unique_adm_year_day.groupby('chartyear')['hadm_id'].nunique()
cumulative_sum = unique_hadm_by_year.cumsum()
total_sum = cumulative_sum.iloc[-1]
split_index = cumulative_sum[cumulative_sum >= 0.8 * total_sum].index[0]
print("split year is",split_index)

split year is 2178


In [47]:
Row_Train = unique_adm_year_day['chartyear']<=split_index
Train_inx = Row_Train[Row_Train].index
Row_Test = unique_adm_year_day['chartyear']>split_index
Test_inx = Row_Test[Row_Test].index

In [ ]:
#array = np.genfromtxt('filename.txt', delimiter=' ', usecols=range(10))-- to test later

In [69]:
file_path = 'Labrado/alllabs1000adm/npval.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
npval = pd.read_csv(obj['Body'], header=None, dtype=float)
npval_train = npval.iloc[Train_inx,:100]
npval_test = npval.iloc[Test_inx,:100]

In [71]:
# current implementation: only support numerical values
import numpy as np
import torch, os
from torch import nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import math
import argparse

class MaskEmbed(nn.Module):
    """ record to mask embedding
    """
    def __init__(self, rec_len=25, embed_dim=64, norm_layer=None):
        
        super().__init__()
        self.rec_len = rec_len
        self.proj = nn.Conv1d(1, embed_dim, kernel_size=1, stride=1)
        self.norm = norm_layer(embed_dim) if norm_layer else nn.Identity()

    def forward(self, x):
        B, _, L = x.shape
        # assert(L == self.rec_len, f"Input data width ({L}) doesn't match model ({self.rec_len}).")
        x = self.proj(x)
        x = x.transpose(1, 2)
        x = self.norm(x)
        return x


class ActiveEmbed(nn.Module):
    """ record to mask embedding
    """
    def __init__(self, rec_len=25, embed_dim=64, norm_layer=None):
        
        super().__init__()
        self.rec_len = rec_len
        self.proj = nn.Conv1d(1, embed_dim, kernel_size=1, stride=1)
        self.norm = norm_layer(embed_dim) if norm_layer else nn.Identity()

    def forward(self, x):
        B, _, L = x.shape
        # assert(L == self.rec_len, f"Input data width ({L}) doesn't match model ({self.rec_len}).")
        x = self.proj(x)
        x = torch.sin(x)
        x = x.transpose(1, 2)
        #   x = torch.cat((torch.sin(x), torch.cos(x + math.pi/2)), -1)
        x = self.norm(x)
        return x



def get_1d_sincos_pos_embed(embed_dim, pos, cls_token=False):
    """
    embed_dim: output dimension for each position
    pos: a list of positions to be encoded: size (M,)
    out: (M, D)
    """

    assert embed_dim % 2 == 0
    omega = np.arange(embed_dim // 2, dtype=float)
    omega /= embed_dim / 2.
    omega = 1. / 10000**omega  # (D/2,)

    pos = np.arange(pos)  # (M,)
    out = np.einsum('m,d->md', pos, omega)  # (M, D/2), outer product

    emb_sin = np.sin(out) # (M, D/2)
    emb_cos = np.cos(out) # (M, D/2)

    pos_embed = np.concatenate([emb_sin, emb_cos], axis=1)  # (M, D)

    if cls_token:
        pos_embed = np.concatenate([np.zeros([1, embed_dim]), pos_embed], axis=0)

    return pos_embed


def adjust_learning_rate(optimizer, epoch, lr, min_lr, max_epochs, warmup_epochs):
    """Decay the learning rate with half-cycle cosine after warmup"""
    if epoch < warmup_epochs:
        tmp_lr = lr * epoch / warmup_epochs 
    else:
        tmp_lr = min_lr + (lr - min_lr) * 0.5 * \
            (1. + math.cos(math.pi * (epoch - warmup_epochs) / (max_epochs - warmup_epochs)))
    for param_group in optimizer.param_groups:
        if "lr_scale" in param_group:
            param_group["lr"] = tmp_lr * param_group["lr_scale"]
        else:
            param_group["lr"] = tmp_lr
    return tmp_lr


def get_grad_norm_(parameters, norm_type: float = 2.0) -> torch.Tensor:
    if isinstance(parameters, torch.Tensor):
        parameters = [parameters]
    parameters = [p for p in parameters if p.grad is not None]
    norm_type = float(norm_type)
    if len(parameters) == 0:
        return torch.tensor(0.)
    device = parameters[0].grad.device
    if norm_type == np.inf:
        total_norm = max(p.grad.detach().abs().max().to(device) for p in parameters)
    else:
        total_norm = torch.norm(torch.stack([torch.norm(p.grad.detach(), norm_type).to(device) for p in parameters]), norm_type)
    return total_norm


class NativeScaler:

    state_dict_key = "amp_scaler"
    def __init__(self):
        self._scaler = torch.cuda.amp.GradScaler()

    def __call__(self, loss, optimizer, clip_grad=None, parameters=None, create_graph=False, update_grad=True):
        self._scaler.scale(loss).backward(create_graph=create_graph)
        if update_grad:
            if clip_grad is not None:
                assert parameters is not None
                self._scaler.unscale_(optimizer)  # unscale the gradients of optimizer's assigned params in-place
                norm = torch.nn.utils.clip_grad_norm_(parameters, clip_grad)
            else:
                self._scaler.unscale_(optimizer)
                norm = get_grad_norm_(parameters)
            self._scaler.step(optimizer)
            self._scaler.update()
        else:
            norm = None
        return norm

    def state_dict(self):
        return self._scaler.state_dict()

    def load_state_dict(self, state_dict):
        self._scaler.load_state_dict(state_dict)



class MAEDataset(Dataset):

    def __init__(self, X, M):        
         self.X = X
         self.M = M

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx: int):
        return self.X[idx], self.M[idx]



def get_dataset(dataset : str, path : str):

    if dataset in ['climate', 'compression', 'wine', 'yacht', 'spam', 'letter', 'credit', 'raisin', 'bike', 'obesity', 'airfoil', 'blood', 'yeast', 'health', 'review', 'travel']:
        df = pd.read_csv(os.path.join(path, 'data', dataset + '.csv'))
        last_col = df.columns[-1]
        y = df[last_col]
        X = df.drop(columns=[last_col])
    elif dataset == 'california':
        from sklearn.datasets import fetch_california_housing
        X, y = fetch_california_housing(as_frame=True, return_X_y=True)
    elif dataset == 'diabetes':
        from sklearn.datasets import load_diabetes
        X, y = load_diabetes(as_frame=True, return_X_y=True)
    elif dataset == 'iris':
        # only for testing
        from sklearn.datasets import load_iris
        X, y = load_iris(as_frame=True, return_X_y=True)
        print("loaded iris dataset")

    return X, y


def get_args_parser():
    parser = argparse.ArgumentParser('MAE pre-training', add_help=False)
    parser.add_argument('--dataset', default='california', type=str)
    parser.add_argument('--batch_size', default=64, type=int,
                        help='Batch size per GPU (effective batch size is batch_size * accum_iter * # gpus')
    parser.add_argument('--max_epochs', default=600, type=int)
    parser.add_argument('--accum_iter', default=1, type=int,
                        help='Accumulate gradient iterations (for increasing the effective batch size under memory constraints)')

    # Model parameters
    parser.add_argument('--mask_ratio', default=0.5, type=float, help='Masking ratio (percentage of removed patches).')
    parser.add_argument('--embed_dim', default=32, type=int, help='embedding dimensions')
    parser.add_argument('--depth', default=6, type=int, help='encoder depth')
    parser.add_argument('--decoder_depth', default=4, type=int, help='decoder depth')
    parser.add_argument('--num_heads', default=4, type=int, help='number of heads')
    parser.add_argument('--mlp_ratio', default=4., type=float, help='mlp ratio')
    parser.add_argument('--encode_func', default='linear', type=str, help='encoding function')

    parser.add_argument('--norm_field_loss', default=False,
                        help='Use (per-patch) normalized field as targets for computing loss')
    parser.set_defaults(norm_field_loss=False)

    # Optimizer parameters
    parser.add_argument('--weight_decay', type=float, default=0.05, help='weight decay (default: 0.05)')
    parser.add_argument('--lr', type=float, default=None, metavar='LR', help='learning rate (absolute lr)')
    parser.add_argument('--blr', type=float, default=1e-3, metavar='LR',
                        help='base learning rate: absolute_lr = base_lr * total_batch_size / 256')
    parser.add_argument('--min_lr', type=float, default=1e-5, metavar='LR',
                        help='lower lr bound for cyclic schedulers that hit 0')
    parser.add_argument('--warmup_epochs', type=int, default=40, metavar='N', help='epochs to warmup LR')

    ###### change this path
    parser.add_argument('--path', default='/data/tianyu/remasker/', type=str, help='dataset path')
    parser.add_argument('--exp_name', default='test', type=str, help='experiment name')

    # Dataset parameters
    parser.add_argument('--device', default='cuda', help='device to use for training / testing')
    parser.add_argument('--seed', default=666, type=int)

    parser.add_argument('--overwrite', default=True, help='whether to overwrite default config')
    parser.add_argument('--pin_mem', action='store_false')

    # distributed training parameters
    return parser

if __name__ == '__main__':
    
    X = torch.tensor([[1, 2, 3, 4]], dtype=torch.float32)
    X = X.unsqueeze(1)
    mask_embed = ActiveEmbed(4, 6)
    print(mask_embed(X).shape)


torch.Size([1, 4, 6])


In [2]:
# current implementation: only support numerical values

from functools import partial
from tkinter import E

import torch
import numpy as np
import torch.nn as nn
import pandas as pd
from timm.models.vision_transformer import Block
from utils import MaskEmbed, get_1d_sincos_pos_embed, ActiveEmbed
eps = 1e-6

class MaskedAutoencoder(nn.Module):
    
    """ Masked Autoencoder with Transformer backbone
    """
    
    def __init__(self, rec_len=25, embed_dim=64, depth=4, num_heads=4,
        decoder_embed_dim=64, decoder_depth=2, decoder_num_heads=4,
        mlp_ratio=4., norm_layer=nn.LayerNorm, norm_field_loss=False, encode_func='linear'):
        super().__init__()

        # --------------------------------------------------------------------------
        # MAE encoder specifics
        
        if encode_func == 'active':
            self.mask_embed = ActiveEmbed(rec_len, embed_dim)
        else:
            self.mask_embed = MaskEmbed(rec_len, embed_dim)
        
        self.rec_len = rec_len
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, rec_len + 1, embed_dim), requires_grad=False)  
        
        self.blocks = nn.ModuleList([
            Block(embed_dim, num_heads, mlp_ratio, qkv_bias=True, norm_layer=norm_layer)
            for i in range(depth)])
        self.norm = norm_layer(embed_dim)


        # --------------------------------------------------------------------------

        # --------------------------------------------------------------------------
        # MAE decoder specifics
        self.decoder_embed = nn.Linear(embed_dim, decoder_embed_dim, bias=True)

        self.mask_token = nn.Parameter(torch.zeros(1, 1, decoder_embed_dim))

        self.decoder_pos_embed = nn.Parameter(torch.zeros(1, rec_len + 1, decoder_embed_dim), requires_grad=False)  # fixed sin-cos embedding

        self.decoder_blocks = nn.ModuleList([
            Block(decoder_embed_dim, decoder_num_heads, mlp_ratio, qkv_bias=True, norm_layer=norm_layer)
            for i in range(decoder_depth)])

        self.decoder_norm = norm_layer(decoder_embed_dim)
        self.decoder_pred = nn.Linear(decoder_embed_dim, 1, bias=True)  # decoder to patch
        
        # --------------------------------------------------------------------------

        self.norm_field_loss = norm_field_loss
        self.initialize_weights()


    def initialize_weights(self):
        
        # initialization
        
        # initialize (and freeze) pos_embed by sin-cos embedding
        pos_embed = get_1d_sincos_pos_embed(self.pos_embed.shape[-1], self.mask_embed.rec_len, cls_token=True)
        self.pos_embed.data.copy_(torch.from_numpy(pos_embed).float().unsqueeze(0))

        decoder_pos_embed = get_1d_sincos_pos_embed(self.decoder_pos_embed.shape[-1], self.mask_embed.rec_len, cls_token=True)
        self.decoder_pos_embed.data.copy_(torch.from_numpy(decoder_pos_embed).float().unsqueeze(0))

        # initialize patch_embed like nn.Linear (instead of nn.Conv2d)
        w = self.mask_embed.proj.weight.data
        torch.nn.init.xavier_uniform_(w.view([w.shape[0], -1]))

        # timm's trunc_normal_(std=.02) is effectively normal_(std=0.02) as cutoff is too big (2.)
        torch.nn.init.normal_(self.cls_token, std=.02)
        torch.nn.init.normal_(self.mask_token, std=.02)

        # initialize nn.Linear and nn.LayerNorm
        self.apply(self._init_weights)


    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            # we use xavier_uniform following official JAX ViT:
            torch.nn.init.xavier_uniform_(m.weight)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)


    def random_masking(self, x, m, mask_ratio):
        """
        Perform per-sample random masking by per-sample shuffling.
        Per-sample shuffling is done by argsort random noise.
        x: [N, L, D], sequence
        """
        N, L, D = x.shape  # batch, length, dim
        if self.training:
            len_keep = int(L * (1 - mask_ratio))
        else:
            len_keep = int(torch.min(torch.sum(m, dim=1)))

        noise = torch.rand(N, L, device=x.device)  # noise in [0, 1]
        noise[m < eps] = 1

        # sort noise for each sample
        ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
        ids_restore = torch.argsort(ids_shuffle, dim=1)

        # keep the first subset
        ids_keep = ids_shuffle[:, :len_keep]

        x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, D))
        
        # generate the binary mask: 0 is keep, 1 is remove
        mask = torch.ones([N, L], device=x.device)
        mask[:, :len_keep] = 0
        # unshuffle to get the binary mask
        mask = torch.gather(mask, dim=1, index=ids_restore)
        nask = torch.ones([N, L], device=x.device) - mask

        if self.training:
            mask[m < eps] = 0

        return x_masked, mask, nask, ids_restore


    def forward_encoder(self, x, m, mask_ratio=0.5):
        
        # embed patches
        x = self.mask_embed(x)

        # add pos embed w/o cls token
        x = x + self.pos_embed[:, 1:, :]    

        # masking: length -> length * mask_ratio
        x, mask, nask, ids_restore = self.random_masking(x, m, mask_ratio)

        # append cls token
        cls_token = self.cls_token + self.pos_embed[:, :1, :]
        cls_tokens = cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # apply Transformer blocks
        for blk in self.blocks:
            x = blk(x)

        x = self.norm(x)

        return x, mask, nask, ids_restore


    def forward_decoder(self, x, ids_restore):
        
        # embed tokens
        x = self.decoder_embed(x)

        # append mask tokens to sequence
        mask_tokens = self.mask_token.repeat(x.shape[0], ids_restore.shape[1] + 1 - x.shape[1], 1)

        x_ = torch.cat([x[:, 1:, :], mask_tokens], dim=1)  # no cls token
        x_ = torch.gather(x_, dim=1, index=ids_restore.unsqueeze(-1).repeat(1, 1, x.shape[2]))  # unshuffle

        x = torch.cat([x[:, :1, :], x_], dim=1)  # append cls token

        # add pos embed
        x = x + self.decoder_pos_embed

        # apply Transformer blocks
        for blk in self.decoder_blocks:
            x = blk(x)
        x = self.decoder_norm(x)

        # predictor projection
        # x = self.decoder_pred(x)
        x = torch.tanh(self.decoder_pred(x))/2 + 0.5

        # remove cls token
        x = x[:, 1:, :]
    
        return x


    def forward_loss(self, data, pred, mask, nask):
        """
        data: [N, 1, L]
        pred: [N, L]
        mask: [N, L], 0 is keep, 1 is remove, 
        """
        # target = self.patchify(data)
        target = data.squeeze(dim=1)
        if self.norm_field_loss:
            mean = target.mean(dim=-1, keepdim=True)
            var = target.var(dim=-1, keepdim=True)
            target = (target - mean) / (var + eps)**.5
        
        
        loss = (pred.squeeze(dim=2) - target) ** 2
        loss = (loss * mask).sum() / mask.sum()  + (loss * nask).sum() / nask.sum()
        # mean loss on removed patches
        
        return loss


    def forward(self, data, miss_idx, mask_ratio=0.5):
        
        latent, mask, nask, ids_restore = self.forward_encoder(data, miss_idx, mask_ratio)
        pred = self.forward_decoder(latent, ids_restore) 
        loss = self.forward_loss(data, pred, mask, nask)
        return loss, pred, mask, nask


def mae_base(**kwargs):
    model = MaskedAutoencoder(
        embed_dim=64, depth=8, num_heads=4,
        decoder_embed_dim=64, decoder_depth=4, decoder_num_heads=4,
        mlp_ratio=2., norm_layer=partial(nn.LayerNorm, eps=eps), **kwargs)
    return model


def mae_medium(**kwargs):
    model = MaskedAutoencoder(
        embed_dim=32, depth=4, num_heads=4,
        decoder_embed_dim=32, decoder_depth=4, decoder_num_heads=4,
        mlp_ratio=4., norm_layer=partial(nn.LayerNorm, eps=eps), **kwargs)
    return model


def mae_large(**kwargs):
    model = MaskedAutoencoder(
        embed_dim=64, depth=8, num_heads=4,
        decoder_embed_dim=64, decoder_depth=4, decoder_num_heads=4,
        mlp_ratio=4., norm_layer=partial(nn.LayerNorm, eps=eps), **kwargs)
    return model


if __name__ == '__main__':

    model = MaskedAutoencoder(
        rec_len=4, embed_dim=8, depth=1, num_heads=1,
        decoder_embed_dim=8, decoder_depth=1, decoder_num_heads=1,
        mlp_ratio=4., norm_layer=partial(nn.LayerNorm, eps=eps)
    )
    
    X = pd.DataFrame([[np.nan, 0.5, np.nan, 0.8]])

    X = torch.tensor(X.values, dtype=torch.float32)
    M = 1 - (1 * (np.isnan(X)))
    X = torch.nan_to_num(X)
    
    X = X.unsqueeze(dim=1)
    print(model.forward(X, M, 0.75))


    


(tensor(0.5094, grad_fn=<AddBackward0>), tensor([[[0.8098],
         [0.8507],
         [0.1312],
         [0.1784]]], grad_fn=<SliceBackward0>), tensor([[0., 0., 0., 1.]]), tensor([[0., 1., 0., 0.]]))


In [75]:
# stdlib
from typing import Any, List, Tuple, Union

# third party
import numpy as np
import math, sys, argparse
import pandas as pd
import torch
from torch import nn
from functools import partial
import time, os, json
from utils import NativeScaler, MAEDataset, adjust_learning_rate, get_dataset
import model_mae
from torch.utils.data import DataLoader, RandomSampler
import sys
import timm.optim.optim_factory as optim_factory
from utils import get_args_parser

# hyperimpute absolute
from hyperimpute.plugins.imputers import ImputerPlugin
from sklearn.datasets import load_iris
from hyperimpute.utils.benchmarks import compare_models
from hyperimpute.plugins.imputers import Imputers
from tqdm import tqdm
eps = 1e-8
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

class ReMasker:

    def __init__(self):
        args = get_args_parser().parse_args()

        self.batch_size = args.batch_size
        self.accum_iter = args.accum_iter
        self.min_lr = args.min_lr
        self.norm_field_loss = args.norm_field_loss
        self.weight_decay = args.weight_decay
        self.lr = args.lr
        self.blr = args.blr
        self.warmup_epochs = 20
        self.model = None
        self.norm_parameters = None

        self.embed_dim = args.embed_dim
        self.depth = args.depth
        self.decoder_depth = args.decoder_depth
        self.num_heads = args.num_heads
        self.mlp_ratio = args.mlp_ratio
        self.max_epochs = 50
        self.mask_ratio = 0.5
        self.encode_func = args.encode_func

    def fit(self, X_raw: pd.DataFrame):

        X = X_raw.copy()

        # Parameters
        no = len(X)
        dim = X.shape[1]

        # X = X.cpu()

        min_val = np.zeros(dim)
        max_val = np.zeros(dim)

# import numpy as np
# import pandas as pd

# Assuming X is a DataFrame and dim is the number of columns
        dim = X.shape[1]
        min_val = np.zeros(dim)
        max_val = np.zeros(dim)
        eps = 1e-7

        for i in range(dim):
            # Use .iloc to access the DataFrame by integer-location
            min_val[i] = np.nanmin(X.iloc[:, i])
            max_val[i] = np.nanmax(X.iloc[:, i])
            # Perform the operation and update the column
            X.iloc[:, i] = (X.iloc[:, i] - min_val[i]) / (max_val[i] - min_val[i] + eps)

        self.norm_parameters = {"min": min_val, "max": max_val}
        np_array = X.to_numpy()

        # Convert NumPy array to PyTorch tensor
        X = torch.tensor(np_array, dtype=torch.float32)
        # Set missing
        M = 1 - (1 * (np.isnan(X)))
        M = M.float().to(device)

        X = torch.nan_to_num(X)
        X = X.to(device)

        self.model = model_mae.MaskedAutoencoder(
            rec_len=dim,
            embed_dim=self.embed_dim,
            depth=self.depth,
            num_heads=self.num_heads,
            decoder_embed_dim=self.embed_dim,
            decoder_depth=self.decoder_depth,
            decoder_num_heads=self.num_heads,
            mlp_ratio=self.mlp_ratio,
            norm_layer=partial(nn.LayerNorm, eps=eps),
            norm_field_loss=self.norm_field_loss,
            encode_func=self.encode_func
        )

        # if self.improve and os.path.exists(self.path):
        #     self.model.load_state_dict(torch.load(self.path))
        #     self.model.to(device)
        #     return self

        self.model.to(device)

        # set optimizers
        # param_groups = optim_factory.add_weight_decay(model, args.weight_decay)
        eff_batch_size = self.batch_size * self.accum_iter
        if self.lr is None:  # only base_lr is specified
            self.lr = self.blr * eff_batch_size / 64
        # param_groups = optim_factory.add_weight_decay(self.model, self.weight_decay)
        # optimizer = torch.optim.AdamW(param_groups, lr=self.lr, betas=(0.9, 0.95))
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr, betas=(0.9, 0.95))
        loss_scaler = NativeScaler()

        dataset = MAEDataset(X, M)
        dataloader = DataLoader(
            dataset, sampler=RandomSampler(dataset),
            batch_size=self.batch_size,
        )

        # if self.resume and os.path.exists(self.path):
        #     self.model.load_state_dict(torch.load(self.path))
        #     self.lr *= 0.5

        self.model.train()

        for epoch in range(self.max_epochs):
            print(epoch)
            optimizer.zero_grad()
            total_loss = 0

            iter = 0

            for iter, (samples, masks) in tqdm(enumerate(dataloader), total = len(dataloader)):

                # we use a per iteration (instead of per epoch) lr scheduler
                if iter % self.accum_iter == 0:
                    adjust_learning_rate(optimizer, iter / len(dataloader) + epoch, self.lr, self.min_lr,
                                         self.max_epochs, self.warmup_epochs)

                samples = samples.unsqueeze(dim=1)
                samples = samples.to(device, non_blocking=True)
                masks = masks.to(device, non_blocking=True)

                # print(samples, masks)

                with torch.cuda.amp.autocast():
                    loss, _, _, _ = self.model(samples, masks, mask_ratio=self.mask_ratio)
                    loss_value = loss.item()
                    total_loss += loss_value

                if not math.isfinite(loss_value):
                    print("Loss is {}, stopping training".format(loss_value))
                    sys.exit(1)

                loss /= self.accum_iter
                loss_scaler(loss, optimizer, parameters=self.model.parameters(),
                            update_grad=(iter + 1) % self.accum_iter == 0)

                if (iter + 1) % self.accum_iter == 0:
                    optimizer.zero_grad()

            total_loss = (total_loss / (iter + 1)) ** 0.5
            # if total_loss < best_loss:
            #     best_loss = total_loss
            #     torch.save(self.model.state_dict(), self.path)
            # if (epoch + 1) % 10 == 0 or epoch == 0:
            # print((epoch+1),',', total_loss)

        # torch.save(self.model.state_dict(), self.path)
        return self

    def transform(self, X_raw: torch.Tensor):
        if not torch.is_tensor(X_raw):
            X_raw = torch.tensor(X_raw.values) 
        X = X_raw.clone()

        min_val = self.norm_parameters["min"]
        max_val = self.norm_parameters["max"]

        no, dim = X.shape
        X = X.cpu()

        # MinMaxScaler normalization
        for i in range(dim):
            X[:, i] = (X[:, i] - min_val[i]) / (max_val[i] - min_val[i] + eps)

        # Set missing
        M = 1 - (1 * (np.isnan(X)))
        X = np.nan_to_num(X)

        X = torch.from_numpy(X).to(device).float()
        M = M.to(device).float()

        self.model.eval()

        # Imputed data
        with torch.no_grad():
            for i in range(no):
                sample = torch.reshape(X[i], (1, 1, -1))
                mask = torch.reshape(M[i], (1, -1))
                _, pred, _, _ = self.model(sample, mask)
                pred = pred.squeeze(dim=2)
                if i == 0:
                    imputed_data = pred
                else:
                    imputed_data = torch.cat((imputed_data, pred), 0)

                    # Renormalize
        for i in range(dim):
            imputed_data[:, i] = imputed_data[:, i] * (max_val[i] - min_val[i] + eps) + min_val[i]

        if np.all(np.isnan(imputed_data.detach().cpu().numpy())):
            err = "The imputed result contains nan. This is a bug. Please report it on the issue tracker."
            raise RuntimeError(err)

        M = M.cpu()
        imputed_data = imputed_data.detach().cpu()
        # print('imputed', imputed_data, M)
        # print('imputed', M * np.nan_to_num(X_raw.cpu()) + (1 - M) * imputed_data)
        return M * np.nan_to_num(X_raw.cpu()) + (1 - M) * imputed_data

    def fit_transform(self, X: torch.Tensor) -> torch.Tensor:
        """Imputes the provided dataset using the GAIN strategy.
        Args:
            X: np.ndarray
                A dataset with missing values.
        Returns:
            Xhat: The imputed dataset.
        """
        X = torch.tensor(X.values, dtype=torch.float32)
        return self.fit(X).transform(X).detach().cpu().numpy()


In [4]:
import numpy as np
import pandas as pd
from utils import get_args_parser
from remasker_impute import ReMasker
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
# X_raw = np.arange(50).reshape(10, 5) * 1.0
# X = pd.DataFrame(X_raw, columns=['0', '1', '2', '3', '4'])
# X.iat[3,0] = np.nan

# imputer = ReMasker()
# print(X)

# imputed = imputer.fit_transform(X)
# print(imputed)
import pandas as pd

In [5]:
import boto3
s3 = boto3.client('s3')
bucket_name = 'sagemaker-studio-905418013525-nvxe84zgs6'
file_path = 'Labrado/alllabs1000adm/alllab1000adm.txt'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
rawdata = obj["Body"].read().decode("utf-8")

In [31]:
lengths = [len(sublist) for sublist in all_data]

In [39]:
pd.DataFrame(lengths).describe()

,0
count,434243.000000
mean,450.466776
std,896.759713
min,0.000000
25%,80.000000
50%,206.000000
75%,479.000000
max,48506.000000


In [24]:
data_lines = rawdata.split('\n')
train = []
test = []
# Iterate through each line and split by spaces
for line in data_lines:
    words = line.split()  # Splits by any whitespace
    year = int(words[1])
    if len(words)>0:
        if year<=2178:
            train.append(words[2:])
        else:
            test.append(words[2:])

IndexError: list index out of range

In [76]:
X_test = npval_test
X_train = npval_train



# # Function to generate new column names
# def generate_new_name(old_name):
#     counter=1
#     mapping = {}
#     # Check if the column name is numeric or has a "_yesterday" suffix
#     base_name = old_name.replace('_yesterday', '')
#     if base_name.isdigit():
#         if base_name not in mapping:
#             # Assign a new testX name and increment the counter
#             mapping[base_name] = f'test{counter}'
#             counter += 1
#         # Return the mapped name, with "_yesterday" suffix if necessary
#         return mapping[base_name] + ('_yesterday' if '_yesterday' in old_name else '')
#     else:
#         # Non-numeric columns remain unchanged
#         return old_name

# # Apply the renaming function to each column
# test.columns = [generate_new_name(col) for col in test.columns]
# X_test = test.filter(regex='^test[1-8](_yesterday)?$') 

# train.columns = [generate_new_name(col) for col in train.columns]
# X_train = train.filter(regex='^test[1-8](_yesterday)?$') 

print("fitting started")
# Initialize your imputer
imputer = ReMasker()

# Since ReMasker is an imputer, we directly fit it on X_train
imputer.fit(X_train)

print("fitting finished")

# Function to mask values in a column
def mask_values(data, column):
    masked_data = data.copy()
    # Randomly mask a certain percentage of the column - adjust as necessary
    mask = np.random.rand(len(masked_data)) < 0.1
    masked_data.loc[mask, column] = np.nan
    return masked_data
print(X_test.columns)
# Evaluate performance for each of the testX columns
for column, column_name in enumerate(X_test.columns):
    print("we are in")
    # Create a copy of X_test with the current column masked
    column=column
    X_test_masked = mask_values(X_test, column_name)
    
    # Impute missing values
    X_test_imputed =  pd.DataFrame(imputer.transform(X_test_masked).cpu().numpy())
    
    print("sucessfully imputed?")

    thingOne=X_test_imputed.iloc[:,column].dropna()

    thingOne.to_csv(f"{column_name}imputed.csv")
    thingTwo=X_test.iloc[:,column].dropna()
    thingTwo.to_csv(f"{column_name}.csv")
    # print(X_test.shape,X_test_imputed.shape)
    # Calculate RMSE, MAE, and R2 for the imputed column
    rmse = sqrt(mean_squared_error(X_test.iloc[:,column].dropna(), X_test_imputed.iloc[:,column].dropna()))
    mae = mean_absolute_error(X_test.iloc[:,column].dropna(), X_test_imputed.iloc[:,column].dropna())
    r2 = r2_score(X_test.iloc[:,column].dropna(), X_test_imputed.iloc[:,column].dropna())
    
    print(f"Evaluation for {column_name}: RMSE = {rmse}, MAE = {mae}, R2 = {r2}")
with open('evaluation_results_yesterday.txt', 'w') as file:
    for column, column_name in enumerate(X_test.columns[:8]):
        # Create a copy of X_test with the current column masked
        X_test_masked = X_test.copy()
        X_test_masked.iloc[:,column]=np.nan
        
        # Impute missing values
        X_test_imputed =  pd.DataFrame(imputer.transform(X_test_masked).cpu().numpy())
        


        # Calculate RMSE, MAE, and R2 for the imputed column
        rmse = sqrt(mean_squared_error(X_test.iloc[:, column].dropna(), X_test_imputed.iloc[:, column].dropna()))
        mae = mean_absolute_error(X_test.iloc[:, column].dropna(), X_test_imputed.iloc[:, column].dropna())
        r2 = r2_score(X_test.iloc[:, column].dropna(), X_test_imputed.iloc[:, column].dropna())
        
        # Construct the output string
        output_str = f"Evaluation for {column_name}: RMSE = {rmse}, MAE = {mae}, R2 = {r2}\n"
        
        # Write to file and print
        file.write(output_str)
        print(output_str)

fitting started


usage: MAE pre-training [--dataset DATASET] [--batch_size BATCH_SIZE]
                        [--max_epochs MAX_EPOCHS] [--accum_iter ACCUM_ITER]
                        [--mask_ratio MASK_RATIO] [--embed_dim EMBED_DIM]
                        [--depth DEPTH] [--decoder_depth DECODER_DEPTH]
                        [--num_heads NUM_HEADS] [--mlp_ratio MLP_RATIO]
                        [--encode_func ENCODE_FUNC]
                        [--norm_field_loss NORM_FIELD_LOSS]
                        [--weight_decay WEIGHT_DECAY] [--lr LR] [--blr LR]
                        [--min_lr LR] [--warmup_epochs N] [--path PATH]
                        [--exp_name EXP_NAME] [--device DEVICE] [--seed SEED]
                        [--overwrite OVERWRITE] [--pin_mem]
MAE pre-training: error: unrecognized arguments: -f /home/sagemaker-user/.local/share/jupyter/runtime/kernel-d21f0679-d5a2-4802-a731-046601ab1110.json


SystemExit: 2

In [24]:
import pandas as pd
import numpy as np
import boto3
s3 = boto3.client('s3')
bucket_name = 'sagemaker-studio-905418013525-nvxe84zgs6'
file_path = 'Labrado/alllabs1000adm/Labitemlist.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
Labitemlist = pd.read_csv(obj['Body'], header=None,names=['labitem', 'cnt'], dtype=int)

file_path = 'Labrado/alllabs1000adm/unique_adm_year_day.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
unique_adm_year_day = pd.read_csv(obj['Body'], header=None,names=['hadm_id', 'chartyear', 'day','rowinx'], dtype=int)

In [25]:
Labitemlist.shape

(375, 2)

In [27]:
unique_adm_year_day.head()

,hadm_id,chartyear,day,rowinx
0,20000019,2159,0,0
1,20000019,2159,1,1
2,20000019,2159,2,2
3,20000024,2151,0,3
4,20000034,2174,0,4


In [15]:
unique_hadm_by_year = unique_adm_year_day.groupby('chartyear')['hadm_id'].nunique()
cumulative_sum = unique_hadm_by_year.cumsum()
total_sum = cumulative_sum.iloc[-1]
split_index = cumulative_sum[cumulative_sum >= 0.8 * total_sum].index[0]
print("split year is",split_index)

split year is 2178


In [47]:
Row_Train = unique_adm_year_day['chartyear']<=split_index
Train_inx = Row_Train[Row_Train].index
Row_Test = unique_adm_year_day['chartyear']>split_index
Test_inx = Row_Test[Row_Test].index

In [60]:
Train_inx

Index([      0,       1,       2,       3,       4,       5,       6,       7,
             8,       9,
       ...
       1966608, 1966609, 1966610, 1966611, 1966612, 1966613, 1966617, 1966618,
       1966619, 1966620],
      dtype='int64', length=1582939)

In [ ]:
#array = np.genfromtxt('filename.txt', delimiter=' ', usecols=range(10))-- to test later

In [67]:
file_path = 'Labrado/alllabs1000adm/npval.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
npval = pd.read_csv(obj['Body'], header=None, dtype=float)
npval_train = npval.iloc[Train_inx[:1000],:5]
npval_test = npval.iloc[Test_inx[:1000],:5]

In [68]:
file_path = 'Labrado/alllabs1000adm/nptime.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
nptime = pd.read_csv(obj['Body'], header=None, dtype=float)
nptime_train = nptime.iloc[Train_inx[:1000],:5]
nptime_test = nptime.iloc[Test_inx[:1000],:5]

,0,1,2,3,4,5,6,7,8,9,...,365,366,367,368,369,370,371,372,373,374
0,3.5,137.0,103.0,26.5,1.1,24.0,14.0,19.0,169.0,183.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.7,136.0,100.0,28.1,1.2,28.0,12.0,17.0,190.0,195.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.9,138.0,102.0,24.4,0.9,26.0,14.0,14.0,177.0,188.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.2,140.0,104.0,32.1,1.1,25.0,16.0,28.0,88.0,196.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.8,141.0,106.0,30.3,2.3,24.0,11.0,26.0,165.0,157.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
file_path = 'Labrado/alllabs1000adm/nptime.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
nptime = pd.read_csv(obj['Body'], header=None, dtype=float)
nptime_train = nptime.iloc[Train_inx[:1000],:5]
nptime_test = nptime.iloc[Test_inx[:1000],:5]
del nptime

In [64]:
file_path = 'Labrado/alllabs1000adm/npval_last.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
npval_last = pd.read_csv(obj['Body'], header=None, dtype=float)
npval_last_train = npval_last.iloc[Train_inx[:1000],:5]
npval_last_test = npval_last.iloc[Test_inx[:1000],:5]

In [65]:
npval_last.iloc[:5,:]

,0,1,2,3,4,5,6,7,8,9,...,365,366,367,368,369,370,371,372,373,374
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
file_path = 'Labrado/alllabs1000adm/nptime_last.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
nptime_last = pd.read_csv(obj['Body'], header=None, dtype=float)
nptime_last_train = nptime_last.iloc[Train_inx[:1000],:5]
nptime_last_test = nptime_last.iloc[Test_inx[:1000],:5]
del nptime_last

In [59]:
nptime_train

,0,1,2,3,4
0,18.0,18.0,18.0,18.0,18.0
1,18.0,18.0,18.0,18.0,18.0
2,17.0,17.0,17.0,14.0,17.0
3,17.0,17.0,17.0,17.0,17.0
4,17.0,17.0,17.0,17.0,17.0
...,...,...,...,...,...
1245,18.0,18.0,18.0,18.0,18.0
1246,18.0,18.0,18.0,18.0,18.0
1247,NaN,NaN,NaN,16.0,NaN
1248,15.0,15.0,15.0,15.0,15.0


In [50]:
# current implementation: only support numerical values
import numpy as np
import torch, os
from torch import nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import math
import argparse

class MaskEmbed(nn.Module):
    """ record to mask embedding
    """
    def __init__(self, rec_len=25, embed_dim=64, norm_layer=None):
        
        super().__init__()
        self.rec_len = rec_len
        self.proj = nn.Conv1d(1, embed_dim, kernel_size=1, stride=1)
        self.norm = norm_layer(embed_dim) if norm_layer else nn.Identity()

    def forward(self, x):
        B, _, L = x.shape
        # assert(L == self.rec_len, f"Input data width ({L}) doesn't match model ({self.rec_len}).")
        x = self.proj(x)
        x = x.transpose(1, 2)
        x = self.norm(x)
        return x


class ActiveEmbed(nn.Module):
    """ record to mask embedding
    """
    def __init__(self, rec_len=25, embed_dim=64, norm_layer=None):
        
        super().__init__()
        self.rec_len = rec_len
        self.proj = nn.Conv1d(1, embed_dim, kernel_size=1, stride=1)
        self.norm = norm_layer(embed_dim) if norm_layer else nn.Identity()

    def forward(self, x):
        B, _, L = x.shape
        # assert(L == self.rec_len, f"Input data width ({L}) doesn't match model ({self.rec_len}).")
        x = self.proj(x)
        x = torch.sin(x)
        x = x.transpose(1, 2)
        #   x = torch.cat((torch.sin(x), torch.cos(x + math.pi/2)), -1)
        x = self.norm(x)
        return x



def get_1d_sincos_pos_embed(embed_dim, pos, cls_token=False):
    """
    embed_dim: output dimension for each position
    pos: a list of positions to be encoded: size (M,)
    out: (M, D)
    """

    assert embed_dim % 2 == 0
    omega = np.arange(embed_dim // 2, dtype=float)
    omega /= embed_dim / 2.
    omega = 1. / 10000**omega  # (D/2,)

    pos = np.arange(pos)  # (M,)
    out = np.einsum('m,d->md', pos, omega)  # (M, D/2), outer product

    emb_sin = np.sin(out) # (M, D/2)
    emb_cos = np.cos(out) # (M, D/2)

    pos_embed = np.concatenate([emb_sin, emb_cos], axis=1)  # (M, D)

    if cls_token:
        pos_embed = np.concatenate([np.zeros([1, embed_dim]), pos_embed], axis=0)

    return pos_embed


def adjust_learning_rate(optimizer, epoch, lr, min_lr, max_epochs, warmup_epochs):
    """Decay the learning rate with half-cycle cosine after warmup"""
    if epoch < warmup_epochs:
        tmp_lr = lr * epoch / warmup_epochs 
    else:
        tmp_lr = min_lr + (lr - min_lr) * 0.5 * \
            (1. + math.cos(math.pi * (epoch - warmup_epochs) / (max_epochs - warmup_epochs)))
    for param_group in optimizer.param_groups:
        if "lr_scale" in param_group:
            param_group["lr"] = tmp_lr * param_group["lr_scale"]
        else:
            param_group["lr"] = tmp_lr
    return tmp_lr


def get_grad_norm_(parameters, norm_type: float = 2.0) -> torch.Tensor:
    if isinstance(parameters, torch.Tensor):
        parameters = [parameters]
    parameters = [p for p in parameters if p.grad is not None]
    norm_type = float(norm_type)
    if len(parameters) == 0:
        return torch.tensor(0.)
    device = parameters[0].grad.device
    if norm_type == np.inf:
        total_norm = max(p.grad.detach().abs().max().to(device) for p in parameters)
    else:
        total_norm = torch.norm(torch.stack([torch.norm(p.grad.detach(), norm_type).to(device) for p in parameters]), norm_type)
    return total_norm


class NativeScaler:

    state_dict_key = "amp_scaler"
    def __init__(self):
        self._scaler = torch.cuda.amp.GradScaler()

    def __call__(self, loss, optimizer, clip_grad=None, parameters=None, create_graph=False, update_grad=True):
        self._scaler.scale(loss).backward(create_graph=create_graph)
        if update_grad:
            if clip_grad is not None:
                assert parameters is not None
                self._scaler.unscale_(optimizer)  # unscale the gradients of optimizer's assigned params in-place
                norm = torch.nn.utils.clip_grad_norm_(parameters, clip_grad)
            else:
                self._scaler.unscale_(optimizer)
                norm = get_grad_norm_(parameters)
            self._scaler.step(optimizer)
            self._scaler.update()
        else:
            norm = None
        return norm

    def state_dict(self):
        return self._scaler.state_dict()

    def load_state_dict(self, state_dict):
        self._scaler.load_state_dict(state_dict)



class MAEDataset(Dataset):

    def __init__(self, X, M):        
         self.X = X
         self.M = M

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx: int):
        return self.X[idx], self.M[idx]



def get_dataset(dataset : str, path : str):

    if dataset in ['climate', 'compression', 'wine', 'yacht', 'spam', 'letter', 'credit', 'raisin', 'bike', 'obesity', 'airfoil', 'blood', 'yeast', 'health', 'review', 'travel']:
        df = pd.read_csv(os.path.join(path, 'data', dataset + '.csv'))
        last_col = df.columns[-1]
        y = df[last_col]
        X = df.drop(columns=[last_col])
    elif dataset == 'california':
        from sklearn.datasets import fetch_california_housing
        X, y = fetch_california_housing(as_frame=True, return_X_y=True)
    elif dataset == 'diabetes':
        from sklearn.datasets import load_diabetes
        X, y = load_diabetes(as_frame=True, return_X_y=True)
    elif dataset == 'iris':
        # only for testing
        from sklearn.datasets import load_iris
        X, y = load_iris(as_frame=True, return_X_y=True)
        print("loaded iris dataset")

    return X, y


def get_args_parser():
    parser = argparse.ArgumentParser('MAE pre-training', add_help=False)
    parser.add_argument('--dataset', default='california', type=str)
    parser.add_argument('--batch_size', default=64, type=int,
                        help='Batch size per GPU (effective batch size is batch_size * accum_iter * # gpus')
    parser.add_argument('--max_epochs', default=600, type=int)
    parser.add_argument('--accum_iter', default=1, type=int,
                        help='Accumulate gradient iterations (for increasing the effective batch size under memory constraints)')

    # Model parameters
    parser.add_argument('--mask_ratio', default=0.5, type=float, help='Masking ratio (percentage of removed patches).')
    parser.add_argument('--embed_dim', default=32, type=int, help='embedding dimensions')
    parser.add_argument('--depth', default=6, type=int, help='encoder depth')
    parser.add_argument('--decoder_depth', default=4, type=int, help='decoder depth')
    parser.add_argument('--num_heads', default=4, type=int, help='number of heads')
    parser.add_argument('--mlp_ratio', default=4., type=float, help='mlp ratio')
    parser.add_argument('--encode_func', default='linear', type=str, help='encoding function')

    parser.add_argument('--norm_field_loss', default=False,
                        help='Use (per-patch) normalized field as targets for computing loss')
    parser.set_defaults(norm_field_loss=False)

    # Optimizer parameters
    parser.add_argument('--weight_decay', type=float, default=0.05, help='weight decay (default: 0.05)')
    parser.add_argument('--lr', type=float, default=None, metavar='LR', help='learning rate (absolute lr)')
    parser.add_argument('--blr', type=float, default=1e-3, metavar='LR',
                        help='base learning rate: absolute_lr = base_lr * total_batch_size / 256')
    parser.add_argument('--min_lr', type=float, default=1e-5, metavar='LR',
                        help='lower lr bound for cyclic schedulers that hit 0')
    parser.add_argument('--warmup_epochs', type=int, default=40, metavar='N', help='epochs to warmup LR')

    ###### change this path
    parser.add_argument('--path', default='/data/tianyu/remasker/', type=str, help='dataset path')
    parser.add_argument('--exp_name', default='test', type=str, help='experiment name')

    # Dataset parameters
    parser.add_argument('--device', default='cuda', help='device to use for training / testing')
    parser.add_argument('--seed', default=666, type=int)

    parser.add_argument('--overwrite', default=True, help='whether to overwrite default config')
    parser.add_argument('--pin_mem', action='store_false')

    # distributed training parameters
    return parser

if __name__ == '__main__':
    
    X = torch.tensor([[1, 2, 3, 4]], dtype=torch.float32)
    X = X.unsqueeze(1)
    mask_embed = ActiveEmbed(4, 6)
    print(mask_embed(X).shape)


torch.Size([1, 4, 6])


In [2]:
# current implementation: only support numerical values

from functools import partial
from tkinter import E

import torch
import numpy as np
import torch.nn as nn
import pandas as pd
from timm.models.vision_transformer import Block
from utils import MaskEmbed, get_1d_sincos_pos_embed, ActiveEmbed
eps = 1e-6

class MaskedAutoencoder(nn.Module):
    
    """ Masked Autoencoder with Transformer backbone
    """
    
    def __init__(self, rec_len=25, embed_dim=64, depth=4, num_heads=4,
        decoder_embed_dim=64, decoder_depth=2, decoder_num_heads=4,
        mlp_ratio=4., norm_layer=nn.LayerNorm, norm_field_loss=False, encode_func='linear'):
        super().__init__()

        # --------------------------------------------------------------------------
        # MAE encoder specifics
        
        if encode_func == 'active':
            self.mask_embed = ActiveEmbed(rec_len, embed_dim)
        else:
            self.mask_embed = MaskEmbed(rec_len, embed_dim)
        
        self.rec_len = rec_len
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, rec_len + 1, embed_dim), requires_grad=False)  
        
        self.blocks = nn.ModuleList([
            Block(embed_dim, num_heads, mlp_ratio, qkv_bias=True, norm_layer=norm_layer)
            for i in range(depth)])
        self.norm = norm_layer(embed_dim)


        # --------------------------------------------------------------------------

        # --------------------------------------------------------------------------
        # MAE decoder specifics
        self.decoder_embed = nn.Linear(embed_dim, decoder_embed_dim, bias=True)

        self.mask_token = nn.Parameter(torch.zeros(1, 1, decoder_embed_dim))

        self.decoder_pos_embed = nn.Parameter(torch.zeros(1, rec_len + 1, decoder_embed_dim), requires_grad=False)  # fixed sin-cos embedding

        self.decoder_blocks = nn.ModuleList([
            Block(decoder_embed_dim, decoder_num_heads, mlp_ratio, qkv_bias=True, norm_layer=norm_layer)
            for i in range(decoder_depth)])

        self.decoder_norm = norm_layer(decoder_embed_dim)
        self.decoder_pred = nn.Linear(decoder_embed_dim, 1, bias=True)  # decoder to patch
        
        # --------------------------------------------------------------------------

        self.norm_field_loss = norm_field_loss
        self.initialize_weights()


    def initialize_weights(self):
        
        # initialization
        
        # initialize (and freeze) pos_embed by sin-cos embedding
        pos_embed = get_1d_sincos_pos_embed(self.pos_embed.shape[-1], self.mask_embed.rec_len, cls_token=True)
        self.pos_embed.data.copy_(torch.from_numpy(pos_embed).float().unsqueeze(0))

        decoder_pos_embed = get_1d_sincos_pos_embed(self.decoder_pos_embed.shape[-1], self.mask_embed.rec_len, cls_token=True)
        self.decoder_pos_embed.data.copy_(torch.from_numpy(decoder_pos_embed).float().unsqueeze(0))

        # initialize patch_embed like nn.Linear (instead of nn.Conv2d)
        w = self.mask_embed.proj.weight.data
        torch.nn.init.xavier_uniform_(w.view([w.shape[0], -1]))

        # timm's trunc_normal_(std=.02) is effectively normal_(std=0.02) as cutoff is too big (2.)
        torch.nn.init.normal_(self.cls_token, std=.02)
        torch.nn.init.normal_(self.mask_token, std=.02)

        # initialize nn.Linear and nn.LayerNorm
        self.apply(self._init_weights)


    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            # we use xavier_uniform following official JAX ViT:
            torch.nn.init.xavier_uniform_(m.weight)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)


    def random_masking(self, x, m, mask_ratio):
        """
        Perform per-sample random masking by per-sample shuffling.
        Per-sample shuffling is done by argsort random noise.
        x: [N, L, D], sequence
        """
        N, L, D = x.shape  # batch, length, dim
        if self.training:
            len_keep = int(L * (1 - mask_ratio))
        else:
            len_keep = int(torch.min(torch.sum(m, dim=1)))

        noise = torch.rand(N, L, device=x.device)  # noise in [0, 1]
        noise[m < eps] = 1

        # sort noise for each sample
        ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
        ids_restore = torch.argsort(ids_shuffle, dim=1)

        # keep the first subset
        ids_keep = ids_shuffle[:, :len_keep]

        x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, D))
        
        # generate the binary mask: 0 is keep, 1 is remove
        mask = torch.ones([N, L], device=x.device)
        mask[:, :len_keep] = 0
        # unshuffle to get the binary mask
        mask = torch.gather(mask, dim=1, index=ids_restore)
        nask = torch.ones([N, L], device=x.device) - mask

        if self.training:
            mask[m < eps] = 0

        return x_masked, mask, nask, ids_restore


    def forward_encoder(self, x, m, mask_ratio=0.5):
        
        # embed patches
        x = self.mask_embed(x)

        # add pos embed w/o cls token
        x = x + self.pos_embed[:, 1:, :]    

        # masking: length -> length * mask_ratio
        x, mask, nask, ids_restore = self.random_masking(x, m, mask_ratio)

        # append cls token
        cls_token = self.cls_token + self.pos_embed[:, :1, :]
        cls_tokens = cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # apply Transformer blocks
        for blk in self.blocks:
            x = blk(x)

        x = self.norm(x)

        return x, mask, nask, ids_restore


    def forward_decoder(self, x, ids_restore):
        
        # embed tokens
        x = self.decoder_embed(x)

        # append mask tokens to sequence
        mask_tokens = self.mask_token.repeat(x.shape[0], ids_restore.shape[1] + 1 - x.shape[1], 1)

        x_ = torch.cat([x[:, 1:, :], mask_tokens], dim=1)  # no cls token
        x_ = torch.gather(x_, dim=1, index=ids_restore.unsqueeze(-1).repeat(1, 1, x.shape[2]))  # unshuffle

        x = torch.cat([x[:, :1, :], x_], dim=1)  # append cls token

        # add pos embed
        x = x + self.decoder_pos_embed

        # apply Transformer blocks
        for blk in self.decoder_blocks:
            x = blk(x)
        x = self.decoder_norm(x)

        # predictor projection
        # x = self.decoder_pred(x)
        x = torch.tanh(self.decoder_pred(x))/2 + 0.5

        # remove cls token
        x = x[:, 1:, :]
    
        return x


    def forward_loss(self, data, pred, mask, nask):
        """
        data: [N, 1, L]
        pred: [N, L]
        mask: [N, L], 0 is keep, 1 is remove, 
        """
        # target = self.patchify(data)
        target = data.squeeze(dim=1)
        if self.norm_field_loss:
            mean = target.mean(dim=-1, keepdim=True)
            var = target.var(dim=-1, keepdim=True)
            target = (target - mean) / (var + eps)**.5
        
        
        loss = (pred.squeeze(dim=2) - target) ** 2
        loss = (loss * mask).sum() / mask.sum()  + (loss * nask).sum() / nask.sum()
        # mean loss on removed patches
        
        return loss


    def forward(self, data, miss_idx, mask_ratio=0.5):
        
        latent, mask, nask, ids_restore = self.forward_encoder(data, miss_idx, mask_ratio)
        pred = self.forward_decoder(latent, ids_restore) 
        loss = self.forward_loss(data, pred, mask, nask)
        return loss, pred, mask, nask


def mae_base(**kwargs):
    model = MaskedAutoencoder(
        embed_dim=64, depth=8, num_heads=4,
        decoder_embed_dim=64, decoder_depth=4, decoder_num_heads=4,
        mlp_ratio=2., norm_layer=partial(nn.LayerNorm, eps=eps), **kwargs)
    return model


def mae_medium(**kwargs):
    model = MaskedAutoencoder(
        embed_dim=32, depth=4, num_heads=4,
        decoder_embed_dim=32, decoder_depth=4, decoder_num_heads=4,
        mlp_ratio=4., norm_layer=partial(nn.LayerNorm, eps=eps), **kwargs)
    return model


def mae_large(**kwargs):
    model = MaskedAutoencoder(
        embed_dim=64, depth=8, num_heads=4,
        decoder_embed_dim=64, decoder_depth=4, decoder_num_heads=4,
        mlp_ratio=4., norm_layer=partial(nn.LayerNorm, eps=eps), **kwargs)
    return model


if __name__ == '__main__':

    model = MaskedAutoencoder(
        rec_len=4, embed_dim=8, depth=1, num_heads=1,
        decoder_embed_dim=8, decoder_depth=1, decoder_num_heads=1,
        mlp_ratio=4., norm_layer=partial(nn.LayerNorm, eps=eps)
    )
    
    X = pd.DataFrame([[np.nan, 0.5, np.nan, 0.8]])

    X = torch.tensor(X.values, dtype=torch.float32)
    M = 1 - (1 * (np.isnan(X)))
    X = torch.nan_to_num(X)
    
    X = X.unsqueeze(dim=1)
    print(model.forward(X, M, 0.75))


    


(tensor(0.5094, grad_fn=<AddBackward0>), tensor([[[0.8098],
         [0.8507],
         [0.1312],
         [0.1784]]], grad_fn=<SliceBackward0>), tensor([[0., 0., 0., 1.]]), tensor([[0., 1., 0., 0.]]))


In [3]:
# stdlib
from typing import Any, List, Tuple, Union

# third party
import numpy as np
import math, sys, argparse
import pandas as pd
import torch
from torch import nn
from functools import partial
import time, os, json
from utils import NativeScaler, MAEDataset, adjust_learning_rate, get_dataset
import model_mae
from torch.utils.data import DataLoader, RandomSampler
import sys
import timm.optim.optim_factory as optim_factory
from utils import get_args_parser

# hyperimpute absolute
from hyperimpute.plugins.imputers import ImputerPlugin
from sklearn.datasets import load_iris
from hyperimpute.utils.benchmarks import compare_models
from hyperimpute.plugins.imputers import Imputers
from tqdm import tqdm
eps = 1e-8
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

class ReMasker:

    def __init__(self):
        args = get_args_parser().parse_args()

        self.batch_size = args.batch_size
        self.accum_iter = args.accum_iter
        self.min_lr = args.min_lr
        self.norm_field_loss = args.norm_field_loss
        self.weight_decay = args.weight_decay
        self.lr = args.lr
        self.blr = args.blr
        self.warmup_epochs = 20
        self.model = None
        self.norm_parameters = None

        self.embed_dim = args.embed_dim
        self.depth = args.depth
        self.decoder_depth = args.decoder_depth
        self.num_heads = args.num_heads
        self.mlp_ratio = args.mlp_ratio
        self.max_epochs = 50
        self.mask_ratio = 0.5
        self.encode_func = args.encode_func

    def fit(self, X_raw: pd.DataFrame):

        X = X_raw.copy()

        # Parameters
        no = len(X)
        dim = X.shape[1]

        # X = X.cpu()

        min_val = np.zeros(dim)
        max_val = np.zeros(dim)

# import numpy as np
# import pandas as pd

# Assuming X is a DataFrame and dim is the number of columns
        dim = X.shape[1]
        min_val = np.zeros(dim)
        max_val = np.zeros(dim)
        eps = 1e-7

        for i in range(dim):
            # Use .iloc to access the DataFrame by integer-location
            min_val[i] = np.nanmin(X.iloc[:, i])
            max_val[i] = np.nanmax(X.iloc[:, i])
            # Perform the operation and update the column
            X.iloc[:, i] = (X.iloc[:, i] - min_val[i]) / (max_val[i] - min_val[i] + eps)

        self.norm_parameters = {"min": min_val, "max": max_val}
        np_array = X.to_numpy()

        # Convert NumPy array to PyTorch tensor
        X = torch.tensor(np_array, dtype=torch.float32)
        # Set missing
        M = 1 - (1 * (np.isnan(X)))
        M = M.float().to(device)

        X = torch.nan_to_num(X)
        X = X.to(device)

        self.model = model_mae.MaskedAutoencoder(
            rec_len=dim,
            embed_dim=self.embed_dim,
            depth=self.depth,
            num_heads=self.num_heads,
            decoder_embed_dim=self.embed_dim,
            decoder_depth=self.decoder_depth,
            decoder_num_heads=self.num_heads,
            mlp_ratio=self.mlp_ratio,
            norm_layer=partial(nn.LayerNorm, eps=eps),
            norm_field_loss=self.norm_field_loss,
            encode_func=self.encode_func
        )

        # if self.improve and os.path.exists(self.path):
        #     self.model.load_state_dict(torch.load(self.path))
        #     self.model.to(device)
        #     return self

        self.model.to(device)

        # set optimizers
        # param_groups = optim_factory.add_weight_decay(model, args.weight_decay)
        eff_batch_size = self.batch_size * self.accum_iter
        if self.lr is None:  # only base_lr is specified
            self.lr = self.blr * eff_batch_size / 64
        # param_groups = optim_factory.add_weight_decay(self.model, self.weight_decay)
        # optimizer = torch.optim.AdamW(param_groups, lr=self.lr, betas=(0.9, 0.95))
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr, betas=(0.9, 0.95))
        loss_scaler = NativeScaler()

        dataset = MAEDataset(X, M)
        dataloader = DataLoader(
            dataset, sampler=RandomSampler(dataset),
            batch_size=self.batch_size,
        )

        # if self.resume and os.path.exists(self.path):
        #     self.model.load_state_dict(torch.load(self.path))
        #     self.lr *= 0.5

        self.model.train()

        for epoch in range(self.max_epochs):
            print(epoch)
            optimizer.zero_grad()
            total_loss = 0

            iter = 0

            for iter, (samples, masks) in tqdm(enumerate(dataloader), total = len(dataloader)):

                # we use a per iteration (instead of per epoch) lr scheduler
                if iter % self.accum_iter == 0:
                    adjust_learning_rate(optimizer, iter / len(dataloader) + epoch, self.lr, self.min_lr,
                                         self.max_epochs, self.warmup_epochs)

                samples = samples.unsqueeze(dim=1)
                samples = samples.to(device, non_blocking=True)
                masks = masks.to(device, non_blocking=True)

                # print(samples, masks)

                with torch.cuda.amp.autocast():
                    loss, _, _, _ = self.model(samples, masks, mask_ratio=self.mask_ratio)
                    loss_value = loss.item()
                    total_loss += loss_value

                if not math.isfinite(loss_value):
                    print("Loss is {}, stopping training".format(loss_value))
                    sys.exit(1)

                loss /= self.accum_iter
                loss_scaler(loss, optimizer, parameters=self.model.parameters(),
                            update_grad=(iter + 1) % self.accum_iter == 0)

                if (iter + 1) % self.accum_iter == 0:
                    optimizer.zero_grad()

            total_loss = (total_loss / (iter + 1)) ** 0.5
            # if total_loss < best_loss:
            #     best_loss = total_loss
            #     torch.save(self.model.state_dict(), self.path)
            # if (epoch + 1) % 10 == 0 or epoch == 0:
            # print((epoch+1),',', total_loss)

        # torch.save(self.model.state_dict(), self.path)
        return self

    def transform(self, X_raw: torch.Tensor):
        if not torch.is_tensor(X_raw):
            X_raw = torch.tensor(X_raw.values) 
        X = X_raw.clone()

        min_val = self.norm_parameters["min"]
        max_val = self.norm_parameters["max"]

        no, dim = X.shape
        X = X.cpu()

        # MinMaxScaler normalization
        for i in range(dim):
            X[:, i] = (X[:, i] - min_val[i]) / (max_val[i] - min_val[i] + eps)

        # Set missing
        M = 1 - (1 * (np.isnan(X)))
        X = np.nan_to_num(X)

        X = torch.from_numpy(X).to(device).float()
        M = M.to(device).float()

        self.model.eval()

        # Imputed data
        with torch.no_grad():
            for i in range(no):
                sample = torch.reshape(X[i], (1, 1, -1))
                mask = torch.reshape(M[i], (1, -1))
                _, pred, _, _ = self.model(sample, mask)
                pred = pred.squeeze(dim=2)
                if i == 0:
                    imputed_data = pred
                else:
                    imputed_data = torch.cat((imputed_data, pred), 0)

                    # Renormalize
        for i in range(dim):
            imputed_data[:, i] = imputed_data[:, i] * (max_val[i] - min_val[i] + eps) + min_val[i]

        if np.all(np.isnan(imputed_data.detach().cpu().numpy())):
            err = "The imputed result contains nan. This is a bug. Please report it on the issue tracker."
            raise RuntimeError(err)

        M = M.cpu()
        imputed_data = imputed_data.detach().cpu()
        # print('imputed', imputed_data, M)
        # print('imputed', M * np.nan_to_num(X_raw.cpu()) + (1 - M) * imputed_data)
        return M * np.nan_to_num(X_raw.cpu()) + (1 - M) * imputed_data

    def fit_transform(self, X: torch.Tensor) -> torch.Tensor:
        """Imputes the provided dataset using the GAIN strategy.
        Args:
            X: np.ndarray
                A dataset with missing values.
        Returns:
            Xhat: The imputed dataset.
        """
        X = torch.tensor(X.values, dtype=torch.float32)
        return self.fit(X).transform(X).detach().cpu().numpy()


In [4]:
import numpy as np
import pandas as pd
from utils import get_args_parser
from remasker_impute import ReMasker
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
# X_raw = np.arange(50).reshape(10, 5) * 1.0
# X = pd.DataFrame(X_raw, columns=['0', '1', '2', '3', '4'])
# X.iat[3,0] = np.nan

# imputer = ReMasker()
# print(X)

# imputed = imputer.fit_transform(X)
# print(imputed)
import pandas as pd

In [5]:
import boto3
s3 = boto3.client('s3')
bucket_name = 'sagemaker-studio-905418013525-nvxe84zgs6'
file_path = 'Labrado/alllabs1000adm/alllab1000adm.txt'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
rawdata = obj["Body"].read().decode("utf-8")

In [31]:
lengths = [len(sublist) for sublist in all_data]

In [39]:
pd.DataFrame(lengths).describe()

,0
count,434243.000000
mean,450.466776
std,896.759713
min,0.000000
25%,80.000000
50%,206.000000
75%,479.000000
max,48506.000000


In [24]:
data_lines = rawdata.split('\n')
train = []
test = []
# Iterate through each line and split by spaces
for line in data_lines:
    words = line.split()  # Splits by any whitespace
    year = int(words[1])
    if len(words)>0:
        if year<=2178:
            train.append(words[2:])
        else:
            test.append(words[2:])

IndexError: list index out of range

In [30]:
train[0]

['51222',
 '0',
 '9.2',
 '50983',
 '0',
 '137.0',
 '51249',
 '0',
 '34.8',
 '51277',
 '0',
 '12.6',
 '50912',
 '0',
 '1.1',
 '50902',
 '0',
 '103.0',
 '51006',
 '0',
 '19.0',
 '51301',
 '0',
 '15.7',
 '51221',
 '0',
 '26.5',
 '51248',
 '0',
 '29.8',
 '50960',
 '0',
 '1.5',
 '51265',
 '0',
 '183.0',
 '50868',
 '0',
 '14.0',
 '50971',
 '0',
 '3.5',
 '50882',
 '0',
 '24.0',
 '51250',
 '0',
 '86.0',
 '50931',
 '0',
 '169.0',
 '51279',
 '0',
 '3.09',
 '50893',
 '24',
 '8.8',
 '50902',
 '24',
 '100.0',
 '51222',
 '24',
 '9.7',
 '50868',
 '24',
 '12.0',
 '51277',
 '24',
 '12.4',
 '51301',
 '24',
 '8.9',
 '51250',
 '24',
 '86.0',
 '50983',
 '24',
 '136.0',
 '50971',
 '24',
 '3.7',
 '51006',
 '24',
 '17.0',
 '50931',
 '24',
 '190.0',
 '51248',
 '24',
 '29.8',
 '51249',
 '24',
 '34.7',
 '51221',
 '24',
 '28.1',
 '50912',
 '24',
 '1.2',
 '50960',
 '24',
 '2.1',
 '51279',
 '24',
 '3.27',
 '50970',
 '24',
 '3.1',
 '51265',
 '24',
 '195.0',
 '50882',
 '24',
 '28.0',
 '51277',
 '49',
 '12.5',
 '50868

In [ ]:
test = pd.read_csv('testYesterdaydata.csv')
train = pd.read_csv('trainYesterdayData.csv')



# Function to generate new column names
def generate_new_name(old_name):
    counter=1
    mapping = {}
    # Check if the column name is numeric or has a "_yesterday" suffix
    base_name = old_name.replace('_yesterday', '')
    if base_name.isdigit():
        if base_name not in mapping:
            # Assign a new testX name and increment the counter
            mapping[base_name] = f'test{counter}'
            counter += 1
        # Return the mapped name, with "_yesterday" suffix if necessary
        return mapping[base_name] + ('_yesterday' if '_yesterday' in old_name else '')
    else:
        # Non-numeric columns remain unchanged
        return old_name

# Apply the renaming function to each column
test.columns = [generate_new_name(col) for col in test.columns]
X_test = test.filter(regex='^test[1-8](_yesterday)?$') 

train.columns = [generate_new_name(col) for col in train.columns]
X_train = train.filter(regex='^test[1-8](_yesterday)?$') 

print("fitting started")
# Initialize your imputer
imputer = ReMasker()

# Since ReMasker is an imputer, we directly fit it on X_train
imputer.fit(X_train)

print("fitting finished")

# Function to mask values in a column
def mask_values(data, column):
    masked_data = data.copy()
    # Randomly mask a certain percentage of the column - adjust as necessary
    mask = np.random.rand(len(masked_data)) < 0.1
    masked_data.loc[mask, column] = np.nan
    return masked_data
print(X_test.columns)
# Evaluate performance for each of the testX columns
for column, column_name in enumerate(X_test.columns):
    print("we are in")
    # Create a copy of X_test with the current column masked
    column=column
    X_test_masked = mask_values(X_test, column_name)
    
    # Impute missing values
    X_test_imputed =  pd.DataFrame(imputer.transform(X_test_masked).cpu().numpy())
    
    print("sucessfully imputed?")

    thingOne=X_test_imputed.iloc[:,column].dropna()

    thingOne.to_csv(f"{column_name}imputed.csv")
    thingTwo=X_test.iloc[:,column].dropna()
    thingTwo.to_csv(f"{column_name}.csv")
    # print(X_test.shape,X_test_imputed.shape)
    # Calculate RMSE, MAE, and R2 for the imputed column
    rmse = sqrt(mean_squared_error(X_test.iloc[:,column].dropna(), X_test_imputed.iloc[:,column].dropna()))
    mae = mean_absolute_error(X_test.iloc[:,column].dropna(), X_test_imputed.iloc[:,column].dropna())
    r2 = r2_score(X_test.iloc[:,column].dropna(), X_test_imputed.iloc[:,column].dropna())
    
    print(f"Evaluation for {column_name}: RMSE = {rmse}, MAE = {mae}, R2 = {r2}")
with open('evaluation_results_yesterday.txt', 'w') as file:
    for column, column_name in enumerate(X_test.columns[:8]):
        # Create a copy of X_test with the current column masked
        X_test_masked = X_test.copy()
        X_test_masked.iloc[:,column]=np.nan
        
        # Impute missing values
        X_test_imputed =  pd.DataFrame(imputer.transform(X_test_masked).cpu().numpy())
        


        # Calculate RMSE, MAE, and R2 for the imputed column
        rmse = sqrt(mean_squared_error(X_test.iloc[:, column].dropna(), X_test_imputed.iloc[:, column].dropna()))
        mae = mean_absolute_error(X_test.iloc[:, column].dropna(), X_test_imputed.iloc[:, column].dropna())
        r2 = r2_score(X_test.iloc[:, column].dropna(), X_test_imputed.iloc[:, column].dropna())
        
        # Construct the output string
        output_str = f"Evaluation for {column_name}: RMSE = {rmse}, MAE = {mae}, R2 = {r2}\n"
        
        # Write to file and print
        file.write(output_str)
        print(output_str)